Descarga de Reportes

In [ ]:
#Condiciones Generales
fecha_de_reporte="29/03/2025"
fecha_inicio=fecha_de_reporte
fecha_fin=fecha_de_reporte

# Librerias
import pandas as pd #Manipulación de datos
import pdfplumber #Leyendo Pdfs
import time #Tiempo
import os #Directorio
from openpyxl import load_workbook # Para trabajar con xlsm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


# Configurar Selenium para forzar descarga automática
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Users\HP\Downloads",  # Ruta donde se guardará automáticamente
    "download.prompt_for_download": False,  # No preguntar dónde guardar
    "download.directory_upgrade": True,  # Actualizar directorio de descargas
    "plugins.always_open_pdf_externally": True  # Evitar que Chrome abra el PDF en el visor
})

driver = webdriver.Chrome(options=options)

# Iniciar sesión
driver.get("https://xn--empeosdelvalle-tnb.com/login/")
usuario = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "usuario")))
contraseña = driver.find_element(By.ID, "password")

# Usuario y contraseñas
usuario.send_keys("")
contraseña.send_keys("")
contraseña.send_keys(Keys.RETURN)

time.sleep(3)

Reporte de Cobranza

In [ ]:
# Ir al reporte
driver.get("https://xn--empeosdelvalle-tnb.com/backoffice/tienda/sistema/184/reporte/reporteprestamocobranza")
time.sleep(3)

# Aplicar filtros
filtro = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "listarpor")))
select = Select(filtro)
select.select_by_visible_text("TODO")

driver.find_element(By.ID, "fechainicio").send_keys(fecha_inicio)
driver.find_element(By.ID, "fechafin").send_keys(fecha_fin)

boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Filtrar reporte')]"))
)
driver.execute_script("arguments[0].click();", boton_buscar)

# Verificar si el botón de descarga está en un iframe
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)  # Cambiar al iframe

# Descargar el PDF
boton_abrir = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "open-button"))
)
boton_abrir.click()

# Esperar para asegurarse de que la descarga se complete
time.sleep(5)

# Abrir el PDF y extraer tablas
ruta = "C:/Users/HP/Downloads/REPORTE_DE_COBRANZA.pdf"  # Ruta del PDF
dataframes = [] #lista vacia para obetener todos los dataframes
with pdfplumber.open(ruta) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            df = pd.DataFrame(tables[1:], columns=tables[0])  # Convertir a DataFrame
            dataframes.append(df)
df_cobranza = pd.concat(dataframes, ignore_index=True)

# Eliminar Archivo
ruta = "C:/Users/HP/Downloads/REPORTE_DE_COBRANZA.pdf"
os.remove(ruta)

# Limpieza de datos
df_cobranza=df_cobranza.rename(columns={"FECHA DE\nCOBRANZA":"FECHA_COBRANZA",
                    "CÓDIGO DE\nCOBRANZA":"CODIGO_COBRANZA",
                    "CÓDIGO DE\nCRÉDITO":"ID_CREDITO",
                    "TOTAL\nCOBRANZA":"TOTAL_COBRANZA"})
df_cobranza["DNI"] = df_cobranza["CLIENTE"].str.replace(r"\D", "", regex=True) #Eliminando todo lo que no es digito
df_cobranza["DNI"]=df_cobranza["DNI"].astype(str)
df_cobranza["CLIENTE"]=df_cobranza["CLIENTE"].str[9:]
df_cobranza=df_cobranza[:-1]
df_cobranza["CAJERO"]=df_cobranza["CAJERO"].str.replace("\n"," ")
df_cobranza["FECHA_COBRANZA"]=df_cobranza["FECHA_COBRANZA"].str[:10]
df_cobranza["FECHA_COBRANZA"] = pd.to_datetime(df_cobranza["FECHA_COBRANZA"], format="%d/%m/%Y").dt.date
df_cobranza["TOTAL_COBRANZA"]=pd.to_numeric(df_cobranza["TOTAL_COBRANZA"])
col = df_cobranza.pop("DNI")  # Extraer la columna "C"
df_cobranza.insert(1, "DNI", col)  # Insertarla en la posición 1

Créditos Cancelados

In [ ]:
# Ir al reporte
driver.get("https://xn--empeosdelvalle-tnb.com/backoffice/tienda/sistema/184/reporte/reporteprestamocredito")
time.sleep(3)

# Aplicar filtros
filtro = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "listarpor")))
select = Select(filtro)
select.select_by_visible_text("TODO")
filtro = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "estadocredito")))
select = Select(filtro)
select.select_by_visible_text("CANCELADOS")
driver.find_element(By.ID, "fechainicio").send_keys(fecha_inicio)
driver.find_element(By.ID, "fechafin").send_keys(fecha_fin)
boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Filtrar reporte')]"))
)
driver.execute_script("arguments[0].click();", boton_buscar)

# Verificar si el botón de descarga está en un iframe
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)  # Cambiar al iframe

# Descargar el PDF
boton_abrir = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "open-button"))
)
boton_abrir.click()
time.sleep(5)

ruta= "C:/Users/HP/Downloads/REPORTE_DE_PRESTAMOS.pdf"  
# Abrir el PDF y extraer tablas
dataframes = [] #lista vacia para obetener todos los dataframes
with pdfplumber.open(ruta) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            df = pd.DataFrame(tables[1:], columns=tables[0])  # Convertir a DataFrame
            dataframes.append(df)
df_cancelado = pd.concat(dataframes, ignore_index=True)
df_cancelado
ruta= "C:/Users/HP/Downloads/REPORTE_DE_PRESTAMOS.pdf"  # Ruta del PDF
os.remove(ruta) #Eliminando Archivo
df_cancelado=df_cancelado.rename(columns={"CÓDIGO\nCRÉDITO":"ID_CREDITO"})
df_cancelado["TIPO"]="CANCELADO"
df_cancelado=df_cancelado[["ID_CREDITO","TIPO"]]
df_cancelado=df_cancelado[:-1]
df_cobranza=pd.merge(df_cobranza,df_cancelado,how="left",on="ID_CREDITO")

# Carga de archivo
ruta_excel = "D:/GRUPO VALLE/RIESGOS Y COBRANZAS/DIARIO/CREDITOS_VIGENTES.xlsm"
book = load_workbook(ruta_excel, keep_vba=True)
sheet = book["PAGOS"]

# Encontrar la última fila con datos reales en la columna "A"
ultima_fila_real = 1  # Inicializar en 1 por seguridad
for row in range(sheet.max_row, 0, -1):  # Buscar de abajo hacia arriba
    if sheet[f"A{row}"].value is not None:  # Verificar que la celda no esté vacía
        ultima_fila_real = row
        break

nueva_fila = ultima_fila_real + 1  # La siguiente fila disponible
# Escribir fila por fila desde la última fila real
for fila in df_cobranza.itertuples(index=False, name=None):
    sheet.append(fila)
# Guardar y cerrar el archivo
book.save(ruta_excel)
book.close()

Reporte de Desembolso

In [ ]:
# Ir al reporte
driver.get("https://xn--empeosdelvalle-tnb.com/backoffice/tienda/sistema/184/reporte/reporteprestamocredito")
time.sleep(3)

# Aplicar filtros
filtro = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "listarpor")))
select = Select(filtro)
select.select_by_visible_text("TODO")

filtro = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "estadocredito")))
select = Select(filtro)
select.select_by_visible_text("DESEMBOLSADOS")

driver.find_element(By.ID, "fechainicio").send_keys(fecha_inicio)
driver.find_element(By.ID, "fechafin").send_keys(fecha_fin)

boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Filtrar reporte')]"))
)
driver.execute_script("arguments[0].click();", boton_buscar)

# Verificar si el botón de descarga está en un iframe
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)  # Cambiar al iframe

# Descargar el PDF
boton_abrir = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "open-button"))
)
boton_abrir.click()

# Esperar para asegurarse de que la descarga se complete
time.sleep(5)

pdf_path = r"C:\Users\HP\Downloads\REPORTE_DE_PRESTAMOS.pdf"  # Ruta del PDF

# Abrir el PDF y extraer tablas
dataframes = [] #lista vacia para obetener todos los dataframes
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_table()
        if tables:
            df = pd.DataFrame(tables[1:], columns=tables[0])  # Convertir a DataFrame
            dataframes.append(df)
df_prestamos = pd.concat(dataframes, ignore_index=True)

os.remove(pdf_path)

df_prestamos=df_prestamos.rename(columns={"CLIENTE (DNI - APELLIDOS,\nNOMBRES)":"CLIENTE",
                                          "CÓDIGO\nCRÉDITO":"ID_CREDITO",
                                          "FECHA\nDESEM.":"FECHA_DESEM",
                                          "MONTO\nDESEM.":"MONTO_DESEM",
                                          "TOTAL\nINTERES\nPENDI.":"INTERES",
                                          "TOTAL\nPENDI.":"CUOTA"
                                          })
df_prestamos["DNI"]=df_prestamos["CLIENTE"].str.replace(r"\D", "", regex=True)
df_prestamos=df_prestamos[:-1]
df_prestamos=df_prestamos.drop(columns=["Nº","TOTAL\nCANCE.","TOTAL\nINTERES\nCANCE.","TOTAL\nMORA\nCANCE.","DESC\nINTERES"])
df_prestamos["CLIENTE"]=df_prestamos["CLIENTE"].str[11:]
df_prestamos["CLIENTE"]=df_prestamos["CLIENTE"].str.replace("\n"," ")
df_prestamos["MONTO_DESEM"] = df_prestamos["MONTO_DESEM"].astype(float)
df_prestamos["CUOTA"] = pd.to_numeric(df_prestamos["CUOTA"])
df_prestamos["INTERES"] = pd.to_numeric(df_prestamos["INTERES"])
df_prestamos["FECHA_DESEM"] = pd.to_datetime(df_prestamos["FECHA_DESEM"], format="%d/%m/%Y").dt.date


In [ ]:
# Crear un DataFrame vacío para almacenar los datos
df_datos = pd.DataFrame(columns=["FECHA_INICIO", "GARANTIA"])

#Función para obtener los datos
def get_datos(ID_credito):
    global df_datos  # Para modificar df_datos dentro de la función
    try:
        driver.get("https://xn--empeosdelvalle-tnb.com/backoffice/tienda/sistema/184/prestamocobranza/create")
        wait = WebDriverWait(driver, 10)

        # Hacer clic en el campo de selección y buscar ID_CREDITO
        select_box = wait.until(EC.element_to_be_clickable((By.ID, "select2-idcliente-container")))
        select_box.click()

        search_box = wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@class="select2-search__field"]')))
        search_box.send_keys(ID_credito)
        time.sleep(5)
        search_box.send_keys(Keys.RETURN)

        # Obtener la fecha de inicio
        fecha_inicio = wait.until(EC.presence_of_element_located((By.XPATH, '//td[@style="padding: 8px;text-align: right;width: 90px;"]')))
        fecha_inicio = fecha_inicio.text

        # Obtener la garantía
        tab_garantias = wait.until(EC.element_to_be_clickable((By.XPATH, '//li[@id="3"]/a')))
        tab_garantias.click()

        wait.until(EC.presence_of_element_located((By.XPATH, '//td[b[contains(text(), "COD.")]]')))
        td_elemento = driver.find_element(By.XPATH, '//td[b[contains(text(), "COD.")]]')
        texto_completo = td_elemento.text.strip() #Eliminando espacios en blanco
        lineas = texto_completo.split("\n") #Hay tres apartados

        producto = lineas[1].strip() if len(lineas) > 1 else "SIN GARANTIA" #En el indice 1 se encuentra la garantia

        # Agregar datos al DataFrame
        df_datos = pd.concat([df_datos, pd.DataFrame({"FECHA_INICIO": [fecha_inicio], "GARANTIA": [producto]})], ignore_index=True)

    except Exception as e:
        print(f"Error con ID_CREDITO {ID_credito}: {str(e)}")

# Obteniendo la fecha de inicio y la garantía
for ID_credito in df_prestamos["ID_CREDITO"]:
    get_datos(ID_credito)

# Concatenamos nuestros datos
df_prestamos=pd.concat([df_prestamos,df_datos],axis=1)

In [ ]:
from dateutil.relativedelta import relativedelta

# Convertir a datetime (si aún no lo está)
df_prestamos["FECHA_INICIO"] = pd.to_datetime(df_prestamos["FECHA_INICIO"], format="%d/%m/%Y")
# Restar un mes y mantener el mismo día
df_prestamos["FECHA_INICIO"] = df_prestamos["FECHA_INICIO"].apply(lambda x: x - relativedelta(months=1))
df_prestamos["FECHA_INICIO"] = pd.to_datetime(df_prestamos["FECHA_INICIO"], format="%d/%m/%Y").dt.date

# Creando nuevas columnas
df_prestamos["ID_GARANTIA"]=None
df_prestamos["CATEGORIA"]=None
df_prestamos["VALOR_MERCADO"]=None
df_prestamos["RVC"]=None
df_prestamos["TASA"]=df_prestamos["INTERES"]/df_prestamos["MONTO_DESEM"]
df_prestamos["TIPO"]=None

# Lista con el orden correcto de las columnas para almacenar en el excel
orden_columnas = [
    "DNI", "CLIENTE", "ID_CREDITO", "ID_GARANTÍA", "CATEGORIA", "GARANTIA",
    "VALOR_MERCADO", "FECHA_DESEM", "FECHA_INICIO", "RVC", "MONTO_DESEM",
    "INTERES", "CUOTA", "TASA", "TIPO"
]

# Reordenar el DataFrame
df_prestamos = df_prestamos.reindex(columns=orden_columnas)

# Trasladar al excel
from openpyxl import load_workbook

# Cargar el archivo
ruta_excel = "D:/GRUPO VALLE/RIESGOS Y COBRANZAS/DIARIO/CREDITOS_VIGENTES.xlsm"
book = load_workbook(ruta_excel, keep_vba=True)
sheet = book["DESEMBOLSO"]

# Encontrar la última fila con datos reales en la columna "A"
ultima_fila_real = 1  # Inicializar en 1 por seguridad
for row in range(sheet.max_row, 0, -1):  # Buscar de abajo hacia arriba
    if sheet[f"A{row}"].value is not None:  # Verificar que la celda no esté vacía
        ultima_fila_real = row
        break
nueva_fila = ultima_fila_real + 1  # La siguiente fila disponible

# Escribir fila por fila desde la última fila real
for fila in df_prestamos.itertuples(index=False, name=None):
    sheet.append(fila)
# Guardar y cerrar el archivo
book.save(ruta_excel)
book.close()

driver.quit()
